# Artificial Neural Network

### Importing the libraries

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [9]:
tf.__version__

'2.4.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [10]:
data = pd.read_csv('Churn_Modelling.csv')


In [11]:
dataset=data.copy()
dataset.drop(['RowNumber','CustomerId','Surname'],axis=1, inplace=True)
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
#Making a list of variables that contain missing values
vars_with_na=[]
for var in dataset.columns:
    if dataset[var].isnull().sum()>0:
        vars_with_na.append(var)
print(vars_with_na)

[]


In [13]:
X=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1].values

In [14]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [15]:
print(y)

[1 0 1 ... 1 1 0]


In [16]:
# make a list of categorical and numerical variables
num_vars=[]
cat_vars=[]

for var in dataset.columns:
    if dataset[var].dtypes !='O':
        num_vars.append(var)
    else:
        cat_vars.append(var)
    

print('Number of numerical variables: ', len(num_vars))
print('Number of categorical variables: ', len(cat_vars))
print('')
print('numerical variables: ',num_vars)
print('')
print( 'categorical variables: ',cat_vars)

Number of numerical variables:  9
Number of categorical variables:  2

numerical variables:  ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']

categorical variables:  ['Geography', 'Gender']


### Encoding categorical data

Label Encoding the "Gender" column

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,1]= le.fit_transform(X[:,1])
X[:,2]= le.fit_transform(X[:,2])

In [18]:
print(X)

[[619 0 0 ... 1 1 101348.88]
 [608 2 0 ... 0 1 112542.58]
 [502 0 0 ... 1 0 113931.57]
 ...
 [709 0 0 ... 0 1 42085.58]
 [772 1 1 ... 1 0 92888.52]
 [792 0 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [20]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [23]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [24]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [25]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [26]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [31]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Training the ANN on the Training set

In [32]:
ann.fit(X_train, y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.7443 - accuracy: 0.5632
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5170 - accuracy: 0.7903
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4533 - accuracy: 0.8038
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4357 - accuracy: 0.8103
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4412 - accuracy: 0.8063
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4104 - accuracy: 0.8290
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4174 - accuracy: 0.8245
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4020 - accuracy: 0.8344
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3981 - accuracy: 0.8374
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4095 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [33]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


### Predicting the Test set results

In [35]:
y_pred=ann.predict(X_test)
y_pred=y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1506   89]
 [ 195  210]]


0.858